# Working with Fine-Tuning


## Univeral Code Used for the Entire Notebook

Let's set up our libraries and client

In [22]:
# Standard library imports
import json  # Handles JSON data encoding and decoding
import random  # Generates random numbers and makes random selections
import time  # Provides time-related functions
import math  # Offers mathematical functions and constants
from pathlib import Path  # Handles filesystem paths in an object-oriented way
from collections import defaultdict  # Provides a dictionary subclass with default values
import base64  # Provides data encoding and decoding as specified in RFC 3548
import io  # Offers core tools for working with streams
import sys  # Provides access to some variables used or maintained by the interpreter

# Third-party library imports
import numpy as np  # Supports large, multi-dimensional arrays and matrices
import pandas as pd  # Offers data manipulation and analysis tools
import tiktoken  # Handles tokenization for OpenAI models
from openai import OpenAI, RateLimitError  # OpenAI API client and related error



In [2]:
# Initialize the OpenAI client
client = OpenAI()  

## Training Files Validation & Metrics

### Data Loading

In [3]:
# Function to load and print the dataset
def load_and_print_dataset(data_path):
    """
    Load the dataset from a given file path and print initial statistics.
    
    Args:
        data_path (str): Path to the dataset file.
        
    Returns:
        dataset (list): Loaded dataset as a list of dictionaries.
    """
    # Load the dataset
    with open(data_path, 'r', encoding='utf-8') as file:
        dataset = [json.loads(line) for line in file]
    
    # Print initial dataset statistics
    print("Number of examples:", len(dataset))
    print("First example:")
    
    # Print messages from the first example in the dataset
    for message in dataset[0]["messages"]:
        print(message)
    
    return dataset

In [4]:
# Using the function to load and print the dataset
data_path = "./artifacts/marv_fine_tune.jsonl"
dataset = load_and_print_dataset(data_path)

Number of examples: 100
First example:
{'role': 'system', 'content': 'Marv is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the tallest mountain in the world?"}
{'role': 'assistant', 'content': "Mount Everest. It's only the tallest thing on the planet."}


### Format Validation

We can perform a variety of error checks to validate that each conversation in the dataset adheres to the format expected by the fine-tuning API. Errors are categorized based on their nature for easier debugging.

1. **Data Type Check**: Checks whether each entry in the dataset is a dictionary (dict). Error type: `data_type`.

2. **Presence of Message List**: Checks if a `messages` list is present in each entry. Error type: `missing_messages_list`.

3. **Message Keys Check**: Validates that each message in the `messages` list contains the keys `role` and `content`. Error type: `message_missing_key`.

4. **Unrecognized Keys in Messages**: Logs if a message has keys other than `role`, `content`, `weight`, `function_call`, and `name`. Error type: `message_unrecognized_key`.

5. **Role Validation**: Ensures the `role` is one of "system", "user", or "assistant". Error type: `unrecognized_role`.

6. **Content Validation**: Verifies that `content` has textual data and is a string. Error type: `missing_content`.

7. **Assistant Message Presence**: Checks that each conversation has at least one message from the assistant. Error type: `example_missing_assistant_message`.


In [5]:
# Function to check for format errors in our file
def check_format_errors(dataset):
    """
    Check for format errors in the dataset and print the results.
    
    Args:
        dataset (list): The dataset to check.
        
    Returns:
        format_errors (dict): A dictionary containing the count of each type of format error.
    """
    # Dictionary to track format errors
    format_errors = defaultdict(int)
    
    # Iterate through each example in the dataset
    for ex in dataset:
        # Check if the example is a dictionary
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue
        
        # Retrieve the messages list from the example
        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue
        
        # Check each message in the messages list
        for message in messages:
            # Check if required keys are present in the message
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1
            
            # Check for any unrecognized keys in the message
            if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
                format_errors["message_unrecognized_key"] += 1
            
            # Validate the role value in the message
            if message.get("role", None) not in ("system", "user", "assistant", "function"):
                format_errors["unrecognized_role"] += 1
            
            # Check content and function_call in the message
            content = message.get("content", None)
            function_call = message.get("function_call", None)
            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1
        
        # Ensure at least one message from the assistant is present
        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1
    
    # Print the results of the error checks
    if format_errors:
        print("Found possible issues:")
        for key, value in format_errors.items():
            print(f"{key}: {value}")
    else:
        print("No errors found")
    
    return format_errors

In [6]:
# Using the function to check for format errors
format_errors = check_format_errors(dataset)

No errors found


## Token Counting Utilities


### Data Warnings, Token Counts, and Cost Estimation

With some lightweight analysis we can identify potential issues in the dataset, like missing messages, and provide statistical insights into message and token counts.

1. **Missing System/User Messages**: Counts the number of conversations missing a "system" or "user" message. Such messages are critical for defining the assistant's behavior and initiating the conversation.

2. **Number of Messages Per Example**: Summarizes the distribution of the number of messages in each conversation, providing insight into dialogue complexity.

3. **Total Tokens Per Example**: Calculates and summarizes the distribution of the total number of tokens in each conversation. Important for understanding fine-tuning costs.

4. **Tokens in Assistant's Messages**: Calculates the number of tokens in the assistant's messages per conversation and summarizes this distribution. Useful for understanding the assistant's verbosity.

5. **Token Limit Warnings**: Checks if any examples exceed the maximum token limit (16,385 tokens), as such examples will be truncated during fine-tuning, potentially resulting in data loss.


Finally, we estimate the total number of tokens that will be used for fine-tuning, which allows us to approximate the cost. It is worth noting that the duration of the fine-tuning jobs will also increase with the token count.

In [10]:
# Constants
MAX_TOKENS_PER_EXAMPLE = 640
TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

# Automatically get the encoding for a specific model
encoding = tiktoken.encoding_for_model("gpt-4o")


def process_dataset(dataset, num_tokens_from_messages,
                    num_assistant_tokens_from_messages, token_limit=64000):
    """
    Process the dataset and calculate various statistics.

    Args:
        dataset (list): List of examples in the dataset.
        num_tokens_from_messages (function): Function to count tokens in messages.
        num_assistant_tokens_from_messages (function): Function to count assistant tokens.
        token_limit (int): Maximum token limit for conversations.

    Returns:
        tuple: Contains lists of message counts, conversation lengths, and assistant message lengths.
    """
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for i, ex in enumerate(dataset):
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        try:
            convo_lens.append(num_tokens_from_messages(messages))
            assistant_message_lens.append(
                num_assistant_tokens_from_messages(messages)
            )
        except Exception as e:
            print(f"Error processing example {i}:")
            print(f"Messages: {messages}")
            print(f"Error: {str(e)}")
            raise

    n_too_long = sum(l > token_limit for l in convo_lens)

    print_summary(n_missing_system, n_missing_user, n_messages,
                convo_lens, assistant_message_lens, n_too_long, token_limit)
    return n_messages, convo_lens, assistant_message_lens


def print_summary(n_missing_system, n_missing_user, n_messages,
                convo_lens, assistant_message_lens, n_too_long, token_limit):
    """
    Print a summary of the dataset processing results.

    Args:
        n_missing_system (int): Number of examples missing system messages.
        n_missing_user (int): Number of examples missing user messages.
        n_messages (list): List of message counts for each example.
        convo_lens (list): List of conversation lengths in tokens.
        assistant_message_lens (list): List of assistant message lengths in tokens.
        n_too_long (int): Number of conversations exceeding the token limit.
        token_limit (int): Maximum token limit for conversations.
    """
    print("Summary of dataset processing:")
    print(f"Num examples missing system message: {n_missing_system}")
    print(f"Num examples missing user message: {n_missing_user}")
    print(f"Total number of examples: {len(n_messages)}")
    print(f"Average number of messages per example: "
        f"{sum(n_messages) / len(n_messages):.2f}")
    print(f"Average number of total tokens per example: "
        f"{sum(convo_lens) / len(convo_lens):.2f}")
    print(f"Average number of assistant tokens per example: "
        f"{sum(assistant_message_lens) / len(assistant_message_lens):.2f}")
    print(f"{n_too_long} examples may be over the {token_limit} token limit "
        f"and will be truncated during fine-tuning")


def calculate_epochs(n_train_examples):
    """
    Calculate the number of epochs based on the number of training examples.

    Args:
        n_train_examples (int): Number of training examples.

    Returns:
        int: Calculated number of epochs.
    """
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        return min(MAX_DEFAULT_EPOCHS,
                math.ceil(MIN_TARGET_EXAMPLES / n_train_examples))
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        return max(MIN_DEFAULT_EPOCHS,
                   MAX_TARGET_EXAMPLES // n_train_examples)
    return TARGET_EPOCHS


def calculate_billing_tokens(convo_lens):
    """
    Calculate the number of billing tokens in the dataset.

    Args:
        convo_lens (list): List of conversation lengths in tokens.

    Returns:
        int: Total number of billing tokens.
    """
    return sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)


def print_dataset_statistics(n_train_examples, convo_lens):
    """
    Print the dataset statistics and billing information.

    Args:
        n_train_examples (int): Number of training examples.
        convo_lens (list): List of conversation lengths in tokens.
    """
    n_epochs = calculate_epochs(n_train_examples)
    n_billing_tokens = calculate_billing_tokens(convo_lens)

    print(f"Dataset Statistics:")
    print(f"- Number of training examples: {n_train_examples}")
    print(f"- Approximate billable tokens: {n_billing_tokens}")
    print(f"- Default number of epochs: {n_epochs}")
    print(f"- Estimated total billable tokens: {n_epochs * n_billing_tokens}")


def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """
    Calculate the number of tokens in a list of messages.

    Args:
        messages (list): List of message dictionaries.
        tokens_per_message (int): Base tokens per message.
        tokens_per_name (int): Additional tokens for the 'name' field.

    Returns:
        int: Total number of tokens.
    """
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            if key == "content" and value is None:
                continue
            elif key == "function_call":
                num_tokens += len(encoding.encode(json.dumps(value)))
            else:
                try:
                    num_tokens += len(encoding.encode(str(value)))
                except Exception as e:
                    print(f"Error encoding key: {key}, value: {value}, "
                        f"type: {type(value)}")
                    print(f"Error message: {str(e)}")
                    raise
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # Adding 3 tokens for end of sequence
    return num_tokens


def num_assistant_tokens_from_messages(messages):
    """
    Calculate the number of tokens in assistant messages.

    Args:
        messages (list): List of message dictionaries.

    Returns:
        int: Total number of tokens in assistant messages.
    """
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            if message.get("content") is not None:
                num_tokens += len(encoding.encode(str(message["content"])))
            if "function_call" in message:
                num_tokens += len(encoding.encode(json.dumps(message["function_call"])))
    return num_tokens

In [11]:
# Process the dataset and extract relevant information
n_messages, convo_lens, assistant_message_lens = process_dataset(
    dataset,
    num_tokens_from_messages,
    num_assistant_tokens_from_messages
)

# Get the total number of examples in the dataset
n_train_examples = len(dataset)

# Print statistics about the dataset
print_dataset_statistics(n_train_examples, convo_lens)

Summary of dataset processing:
Num examples missing system message: 0
Num examples missing user message: 0
Total number of examples: 100
Average number of messages per example: 3.00
Average number of total tokens per example: 46.01
Average number of assistant tokens per example: 10.54
0 examples may be over the 64000 token limit and will be truncated during fine-tuning
Dataset Statistics:
- Number of training examples: 100
- Approximate billable tokens: 4601
- Default number of epochs: 3
- Estimated total billable tokens: 13803


## Train / Test Split

In [12]:
# Train / Test Split Function for JSONL Files
def split_jsonl_file(file_path, train_ratio=0.8):
    # Read the input file
    file_path = Path(file_path)
    with file_path.open('r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    
    # Shuffle the data
    random.shuffle(data)
    
    # Calculate split index
    split_index = int(len(data) * train_ratio)
    
    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]
    
    # Prepare output file paths
    train_file = file_path.with_name(f"{file_path.stem}_train{file_path.suffix}")
    test_file = file_path.with_name(f"{file_path.stem}_test{file_path.suffix}")
    
    # Write train data
    with train_file.open('w', encoding='utf-8') as f:
        for item in train_data:
            json.dump(item, f)
            f.write('\n')
    
    # Write test data
    with test_file.open('w', encoding='utf-8') as f:
        for item in test_data:
            json.dump(item, f)
            f.write('\n')
    
    print(f"Train data saved to: {train_file}")
    print(f"Test data saved to: {test_file}")
    print(f"Train set size: {len(train_data)}")
    print(f"Test set size: {len(test_data)}")
    
    return(train_file, test_file)



In [13]:
# File paths and data processing
file_path = "./artifacts/marv_fine_tune.jsonl"

# Split the JSONL file into train and test sets
train_test_files = split_jsonl_file(file_path)
print("\n")  # Print a blank line for better output readability

# Convert the returned file paths to strings
train_path, test_path = [str(file) for file in train_test_files]

# Print the paths of the resulting train and test files
print(f"Train file path: {train_path}")
print(f"Test file path: {test_path}")

Train data saved to: artifacts\marv_fine_tune_train.jsonl
Test data saved to: artifacts\marv_fine_tune_test.jsonl
Train set size: 80
Test set size: 20


Train file path: artifacts\marv_fine_tune_train.jsonl
Test file path: artifacts\marv_fine_tune_test.jsonl


## Creating a Fine-Tuning Job

### Uploading Training and Test Files

In [14]:
# Upload the training data to the OpenAI API
train__set_file = client.files.create(
            file=open(train_path, "rb"),
            purpose="fine-tune"
            )

# Upload the test data to the OpenAI API
test_set_file = client.files.create(
            file=open(test_path, "rb"),
            purpose="fine-tune"
            )

### Creating a Simple Fine-Tuning Job (Code)

In [15]:
# Create a fine-tuning job using the uploaded training data
simple_ft_job = client.fine_tuning.jobs.create(
    training_file=train__set_file.id, 
    model="gpt-4o-mini-2024-07-18"
)

### Create a Fine-Tuning Job with All Parameters

In [17]:
# Create a fine-tuning job using the uploaded training data
all_params_ft_job = client.fine_tuning.jobs.create(
    model="gpt-4o-mini-2024-07-18",  # Base model to be fine-tuned
    training_file=train__set_file.id,  # ID of the uploaded training data file
    validation_file=test_set_file.id,  # ID of the uploaded validation (test) data file
    hyperparameters={
        "batch_size": "auto",  # Let API automatically determine batch size
        "learning_rate_multiplier": "auto",  # Auto-set learning rate multiplier
        "n_epochs": "auto",  # Automatically decide number of training epochs
    },
    suffix="marv_ft_0003",  # Append this to the fine-tuned model's name
    integrations=None,  # Specific integrations used
    seed=None,  # Specific random seed set for reproducibility
)

## Analyzing the Training Metrics

### Pulling Training Metrics Data

In [18]:
# This set of code will check the status of the fine-tuning job
# Repeating the check every 60 seconds until the job is done

class FineTuningFailedException(Exception):
    """Custom exception for failed fine-tuning jobs."""
    pass

def check_fine_tuning_status(client, job_id):
    """
    Continuously check the status of a fine-tuning job until it succeeds or fails.

    Args:
        client: The API client object.
        job_id: The ID of the fine-tuning job to check.

    Returns:
        The final job details if the job succeeds.

    Raises:
        FineTuningFailedException: If the fine-tuning job fails.
        Exception: For any other errors during the process.
    """
    while True:
        try:
            # Retrieve updated information for the fine-tuning job
            retrieved_job = client.fine_tuning.jobs.retrieve(job_id)
            
            print(f"Current status: {retrieved_job.status}")
            
            if retrieved_job.status == "failed":
                print("Job failed. Final job details:")
                print(retrieved_job)
                raise FineTuningFailedException("The fine-tuning job has failed.")
            
            if retrieved_job.status == "succeeded":
                print("Job succeeded. Final job details:")
                print(retrieved_job)
                return retrieved_job
            
            # Wait for 60 seconds before checking again
            time.sleep(60)
        
        except Exception as e:
            print(f"An error occurred: {e}")
            raise  # Re-raise the exception to stop the function
        


In [19]:
# Use the check_fine_tuning_status function

# Extract the job ID from the created fine-tuning job
job_id = all_params_ft_job.id

# Monitor the fine-tuning job status until completion
final_job = check_fine_tuning_status(client, job_id)

Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: succeeded
Job succeeded. Final job details:
FineTuningJob(id='ftjob-iNtCqKhdZUJ583nxlavNW3OW', created_at=1723388837, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4nlv', finished_at=1723389392, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=['file-Kka4BsNhocKqPriFlPs7aSGN'], seed=622433556, status='succeeded', trained_tokens=10521, training_file='file-aRcuFvUVcDiFlTLuJ2zuOf42', validation_file='file-l4XN9lwDjLRdfH1yhQammB2N', estimated_finish=None, integrations=[], user_provided_suffix='marv_ft_0003')


In [20]:
# Function to get the training metrics for a fine-tuning job
def fetch_and_process_fine_tuning_metrics(client, fine_tuning_job_id):
    # Function to replace colons with underscores for file names
    def replace_colons_with_underscores(input_string):
        return input_string.replace(':', '_')
    
    # Get the training metrics for the fine-tuning job
    fine_tune_job = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
        
    # File ID of the file you want to download
    file_id = fine_tune_job.result_files[0]
    
    # Retrieve the file content directly
    response = client.files.content(file_id)
    
    # Decode the Base64 content
    decoded_content = base64.b64decode(response.content).decode('utf-8')
    
    # Create a DataFrame from the decoded content
    df = pd.read_csv(io.StringIO(decoded_content))
        
    # Save the CSV file locally:
    metrics_file_name = "step_metrics_" + replace_colons_with_underscores(fine_tune_job.fine_tuned_model + ".csv")
    df.to_csv(metrics_file_name, index=False)
    print(f"\nFile saved as {metrics_file_name}")
    
    # Return our dataframe in case the caller wants to do more with it
    return df

# Example usage
fetch_and_process_fine_tuning_metrics(client, all_params_ft_job .id)




File saved as step_metrics_ft_gpt-4o-mini-2024-07-18_personal_marv-ft-0003_9v4b4nlv.csv


,step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
0,1,3.07854,0.46154,NaN,NaN
1,2,2.32874,0.55000,NaN,NaN
2,3,5.84713,0.57143,NaN,NaN
3,4,4.80234,0.46667,NaN,NaN
4,5,3.73353,0.43750,NaN,NaN
...,...,...,...,...,...
235,236,0.51398,0.84615,NaN,NaN
236,237,0.14378,0.94444,NaN,NaN
237,238,0.01390,1.00000,NaN,NaN
238,239,0.19765,0.92308,NaN,NaN


## Exploring Fine-Tuning Jobs

### List Fine-Tuning Jobs

In [38]:
# list our fine-tuning jobs
ft_jobs_list = client.fine_tuning.jobs.list()

print(ft_jobs_list)
print("\n")

# Print job IDs, objects, and statuses
for job in ft_jobs_list.data:
    print(job.id, job.object, job.status)


# Print detailed information for only the first job in the list
job = ft_jobs_list.data[0]

print("\n")
print(f"""
Job ID: {job.id}
Created At: {job.created_at}
Error: {job.error}
Fine-tuned Model: {job.fine_tuned_model}
Finished At: {job.finished_at}
Hyperparameters:
    - Epochs: {job.hyperparameters.n_epochs}
    - Batch Size: {job.hyperparameters.batch_size}
    - Learning Rate Multiplier: {job.hyperparameters.learning_rate_multiplier}
Model: {job.model}
Object: {job.object}
Organization ID: {job.organization_id}
Result Files: {', '.join(str(file) for file in job.result_files)}
Seed: {job.seed}
Status: {job.status}
Trained Tokens: {job.trained_tokens}
Training File: {job.training_file}
Validation File: {job.validation_file}
Estimated Finish: {job.estimated_finish}
Integrations: {', '.join(str(integration) for integration in job.integrations) if job.integrations else 'None'}
User Provided Suffix: {job.user_provided_suffix}
""")


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-WcdcO2exERwuEIK1fEcM2JBT', created_at=1723400659, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hfQoT', finished_at=1723401333, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=['file-9rvOEZkwsslSW94AI2J0GcBU'], seed=32159879, status='succeeded', trained_tokens=10542, training_file='file-oeSDEG9qQLA9aGSL3wN8yLsO', validation_file='file-4sbdIBCMl6yGsqIpVrjmuA0c', estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='Marv_Fun_Tune_v2', entity='suspicious-cow-self', name=None, tags=None, run_id='ftjob-WcdcO2exERwuEIK1fEcM2JBT'))], user_provided_suffix='marv_wandb_tune'), FineTuningJob(id='ftjob-ueWCuRQ1x3VqEXRAjJ6Kv5Oa', c

### Retrieving Fine-Tuning Jobs

In [40]:
# Retrieve the fine-tuning job by ID
retrieved_job = client.fine_tuning.jobs.retrieve(job.id)

print(f"""
retrieved_job ID: {retrieved_job.id}
Created At: {retrieved_job.created_at}
Error: {retrieved_job.error}
Fine-tuned Model: {retrieved_job.fine_tuned_model}
Finished At: {retrieved_job.finished_at}
Hyperparameters:
    - Epochs: {retrieved_job.hyperparameters.n_epochs}
    - Batch Size: {retrieved_job.hyperparameters.batch_size}
    - Learning Rate Multiplier: {retrieved_job.hyperparameters.learning_rate_multiplier}
Model: {retrieved_job.model}
Object: {retrieved_job.object}
Organization ID: {retrieved_job.organization_id}
Result Files: {', '.join(str(file) for file in job.result_files)}
Seed: {retrieved_job.seed}
Status: {retrieved_job.status}
Trained Tokens: {retrieved_job.trained_tokens}
Training File: {retrieved_job.training_file}
Validation File: {retrieved_job.validation_file}
Estimated Finish: {retrieved_job.estimated_finish}
Integrations: {', '.join(str(integration) for integration in job.integrations) if job.integrations else 'None'}
User Provided Suffix: {retrieved_job.user_provided_suffix}
""")



retrieved_job ID: ftjob-WcdcO2exERwuEIK1fEcM2JBT
Created At: 1723400659
Error: Error(code=None, message=None, param=None)
Fine-tuned Model: ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hfQoT
Finished At: 1723401333
Hyperparameters:
    - Epochs: 3
    - Batch Size: 1
    - Learning Rate Multiplier: 1.8
Model: gpt-4o-mini-2024-07-18
Object: fine_tuning.job
Organization ID: org-SQH2HT1IvRszon9pdYwV1yvQ
Result Files: file-9rvOEZkwsslSW94AI2J0GcBU
Seed: 32159879
Status: succeeded
Trained Tokens: 10542
Training File: file-oeSDEG9qQLA9aGSL3wN8yLsO
Validation File: file-4sbdIBCMl6yGsqIpVrjmuA0c
Estimated Finish: None
Integrations: FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='Marv_Fun_Tune_v2', entity='suspicious-cow-self', name=None, tags=None, run_id='ftjob-WcdcO2exERwuEIK1fEcM2JBT'))
User Provided Suffix: marv_wandb_tune



## Understanding Checkpoints

### Listing Job Checkpoints

In [43]:
# List all fine-tuning job checkpoints
ft_jobs_checkpoints_list = client.fine_tuning.jobs.checkpoints.list(all_params_ft_job.id)

# Print the entire list of checkpoints
print(ft_jobs_checkpoints_list)
print("\n\n")  # Add two blank lines for better readability

# Iterate through each checkpoint and print specific details
for checkpoint in ft_jobs_checkpoints_list:
    print("Checkpoint ID: ", checkpoint.id, "\n",                        # Unique identifier for the checkpoint
        "Fine-tuned Model Checkpoint: ", checkpoint.fine_tuned_model_checkpoint, "\n",  # Checkpoint of the fine-tuned model
        "Step Number: ", checkpoint.step_number, "\n",                # Training step at which checkpoint was saved
        "Created At: ", checkpoint.created_at, "\n",                  # Timestamp of checkpoint creation
        "Training Loss: ", checkpoint.metrics.train_loss, "\n",       # Training loss at this checkpoint
        "Validation Loss: ", checkpoint.metrics.valid_loss, "\n",       # Validation loss at this checkpoint
        "Fine-tuning Job ID: ", checkpoint.fine_tuning_job_id, "\n"   # ID of the associated fine-tuning job
        )


SyncCursorPage[FineTuningJobCheckpoint](data=[FineTuningJobCheckpoint(id='ftckpt_RLf3ZqwahdDD2iT4otJErkRr', created_at=1723389359, fine_tuned_model_checkpoint='ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4nlv', fine_tuning_job_id='ftjob-iNtCqKhdZUJ583nxlavNW3OW', metrics=Metrics(full_valid_loss=None, full_valid_mean_token_accuracy=None, step=240.0, train_loss=0.0005891973269172013, train_mean_token_accuracy=1.0, valid_loss=0.879476819719587, valid_mean_token_accuracy=0.7142857142857143), object='fine_tuning.job.checkpoint', step_number=240), FineTuningJobCheckpoint(id='ftckpt_Ig4cmKuYHDJO82NrANrGhj2i', created_at=1723389228, fine_tuned_model_checkpoint='ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4DyO:ckpt-step-160', fine_tuning_job_id='ftjob-iNtCqKhdZUJ583nxlavNW3OW', metrics=Metrics(full_valid_loss=None, full_valid_mean_token_accuracy=None, step=160.0, train_loss=0.4574517607688904, train_mean_token_accuracy=0.9230769276618958, valid_loss=1.620632318349985, valid_mean

## Exploring Fine-tuning Events

### List Fine-Tuning Events

In [45]:
# List the events for the first fine-tuning job in our list
ft_events_list = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=all_params_ft_job.id,
    limit=10  # Limit the number of events to retrieve
)

# Print the entire list of events
print(ft_events_list)
print("\n\n")  # Add two blank lines for better readability

# Iterate through each event and print specific details
for event in ft_events_list.data:
    print("Event ID: ", event.id, "\n",             # Unique identifier for the event
        "Object Type: ", event.object, "\n",      # Type of object (likely 'fine_tuning.job.event')
        "Created At: ", event.created_at, "\n",   # Timestamp when the event was created
        "Level: ", event.level, "\n",             # Importance level of the event (e.g., 'info', 'warning')
        "Type: ", event.type, "\n",               # Type of event
        "Message: ", event.message, "\n"          # Descriptive message about the event
        )


SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-Tqz5wMQryfFZ9swoEe5nVDbc', created_at=1723389401, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-ITIsz5gfqV8Kkx1NnN1Bs687', created_at=1723389395, level='info', message='New fine-tuned model created: ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4nlv', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-cfOMQrpKLekrCF406MhluXay', created_at=1723389395, level='info', message='Checkpoint created at step 160 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4DyO:ckpt-step-160', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-TpucuRtX4ZnmfwWXQnNhUJL9', created_at=1723389395, level='info', message='Checkpoint created at step 80 with Snapshot ID: ft:gpt-4o-mini-2024-07-18:personal:marv-ft-0003:9v4b4tuF:ckpt-step-80', o

### Cancelling Fine-Tuning Jobs

In [46]:
# Create a fine-tuning job using the uploaded training data
dead_ft_job_walking = client.fine_tuning.jobs.create(
        model="gpt-3.5-turbo-0125",
        training_file=train__set_file.id, 
        validation_file=None,
        hyperparameters={
            "batch_size": "auto",
            "learning_rate_multiplier": "auto",
            "n_epochs": "auto",
        },
        suffix="dead_walking",
        integrations=None,
        seed=None,
    )

# Cancel the fine-tuning job
client.fine_tuning.jobs.cancel(dead_ft_job_walking.id)

# Retrieve the fine-tuning job by ID and show the updated status
dead_ft_job_walking_status = client.fine_tuning.jobs.retrieve(dead_ft_job_walking.id)

print(f"Job status after cancellation: {dead_ft_job_walking_status.status}")

Job status after cancellation: cancelled


## Using Fine-Tuned Models

### Using Fine-Tuned Models

In [47]:
# list all our fine-tuning jobs
ft_jobs_list = client.fine_tuning.jobs.list()

# Print job IDs, objects, and statuses for the filtered list
print("===== All Jobs =====")
for job in ft_jobs_list:
    print(job.id, job.object, job.status)

print("\n")

# Filter the list to only include jobs with a status of "succeeded"
succeeded_jobs_list = [job for job in ft_jobs_list.data if job.status == "succeeded"]

print("===== Successful Jobs =====")

# Print job IDs, objects, and statuses for the filtered list
for job in succeeded_jobs_list:
    print(job.id, job.object, job.status)


===== All Jobs =====
ftjob-4DjQalMlp0Ua7Mbciu593HmZ fine_tuning.job cancelled
ftjob-WcdcO2exERwuEIK1fEcM2JBT fine_tuning.job succeeded
ftjob-ueWCuRQ1x3VqEXRAjJ6Kv5Oa fine_tuning.job succeeded
ftjob-ZjHbXqe1ficE0kvMy7RxfJrE fine_tuning.job succeeded
ftjob-iNtCqKhdZUJ583nxlavNW3OW fine_tuning.job succeeded
ftjob-1SJhNwARrgon5LLTLnqG4Dvx fine_tuning.job succeeded
ftjob-V9xgWRDss6V3gp1jpybSXHdn fine_tuning.job succeeded
ftjob-iSjFrat8Pcfeyug3kwL2FM9M fine_tuning.job succeeded
ftjob-oeBH71kUoI5hSabOiCH8k3zS fine_tuning.job succeeded
ftjob-CRVtbKaFJ8mJBraHDHzfezi8 fine_tuning.job succeeded
ftjob-Nyik8wkNII2YoWMtk5cIsqM7 fine_tuning.job succeeded
ftjob-QRBP5PMMaunkG3stIP3JKy3D fine_tuning.job succeeded
ftjob-IXlkUz9Arr7OwcioV5sInNZc fine_tuning.job succeeded
ftjob-Cta6HSxQEq6BKIS69I791jiw fine_tuning.job succeeded
ftjob-RfIjCVMhQu4IqJrOMHJKnKJC fine_tuning.job succeeded
ftjob-LSnA15L6lyVC1SKzPcBjalcK fine_tuning.job succeeded
ftjob-rbSxX1ZOeSaKgoyPfRqMwOSQ fine_tuning.job succeeded
ftjob-anNK

In [48]:
# Use the fine-tuned model to generate a completion
completion = client.chat.completions.create(
    model=succeeded_jobs_list[0].fine_tuned_model,  # Use the first successful fine-tuned model
    messages=[
        {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
        {"role": "user", "content": "What is limburger cheese?"},
    ]
)

# Print the generated response
print(completion.choices[0].message.content)

A pungent cheese that smells worse than your gym socks.


### Using Checkpointed Models

In [49]:
# List all fine-tuning job checkpoints for the first successful job
ft_jobs_checkpoints_list = client.fine_tuning.jobs.checkpoints.list(
    succeeded_jobs_list[0].id  # Use the ID of the first successful job
)

# Iterate through each checkpoint and print specific details
for checkpoint in ft_jobs_checkpoints_list:
    print("Checkpoint ID: ", checkpoint.id, "\n",                            # Unique identifier for the checkpoint
        "Fine-tuned Model Checkpoint: ", checkpoint.fine_tuned_model_checkpoint, "\n",  # Checkpoint of the fine-tuned model
        "Step Number: ", checkpoint.step_number, "\n",                    # Training step at which checkpoint was saved
        "Created At: ", checkpoint.created_at, "\n",                      # Timestamp of checkpoint creation
        "Training Loss: ", checkpoint.metrics.train_loss, "\n",           # Training loss at this checkpoint
        "Fine-tuning Job ID: ", checkpoint.fine_tuning_job_id, "\n"       # ID of the associated fine-tuning job
        )


Checkpoint ID:  ftckpt_tE1N59IDI5xmehycpoSOEwuk 
 Fine-tuned Model Checkpoint:  ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hfQoT 
 Step Number:  240 
 Created At:  1723401300 
 Training Loss:  0.0022328693885356188 
 Fine-tuning Job ID:  ftjob-WcdcO2exERwuEIK1fEcM2JBT 

Checkpoint ID:  ftckpt_YA4ZvnRde7VNlc7fI4k8HBsi 
 Fine-tuned Model Checkpoint:  ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hfiZn:ckpt-step-160 
 Step Number:  160 
 Created At:  1723401150 
 Training Loss:  0.1336524933576584 
 Fine-tuning Job ID:  ftjob-WcdcO2exERwuEIK1fEcM2JBT 

Checkpoint ID:  ftckpt_01FljaJYfZrzv8VsRNq9GsJC 
 Fine-tuned Model Checkpoint:  ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hetnX:ckpt-step-80 
 Step Number:  80 
 Created At:  1723401002 
 Training Loss:  0.612462043762207 
 Fine-tuning Job ID:  ftjob-WcdcO2exERwuEIK1fEcM2JBT 



In [50]:
# Get the first checkpoint from the paginated results
first_checkpoint = next(iter(ft_jobs_checkpoints_list), None)

if first_checkpoint:
    # Use the checkpointed model to generate a completion
    completion = client.chat.completions.create(
        model=first_checkpoint.fine_tuned_model_checkpoint,  # Use the checkpointed model
        messages=[
            {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
            {"role": "user", "content": "What is limburger cheese?"},
        ]
    )
    # Print the generated response
    print(completion.choices[0].message.content)
else:
    print("No checkpoints found.")

A stinky cheese. Literally. It's known for its strong smell and taste.


## Deleting Fine-Tuned Models

In [ ]:
# Delete a fine-tuned model
# Note: This code is currently commented out

# response = client.models.delete("ft:gpt-4o-mini-2024-07-18:personal::9rCgzkVh")
# print(response)

## Fine-Tuning Use Cases

### Structured Output: Sports Headlines

#### Validating Our Data File

In [60]:
# Validate the data file that will be used for fine-tuning

# Path to the JSONL data file
data_path = "./artifacts/sports_headlines.jsonl"

# Load and print the dataset
dataset = load_and_print_dataset(data_path)
print("\n")  # Add a blank line for better readability

# Check for format errors in the dataset
format_errors = check_format_errors(dataset)
print("\n")  # Add a blank line for better readability

# Process the dataset to extract message counts and token lengths
n_messages, convo_lens, assistant_message_lens = process_dataset(
    dataset, 
    num_tokens_from_messages, 
    num_assistant_tokens_from_messages
)
print("\n")  # Add a blank line for better readability

# Get the total number of examples in the dataset
n_train_examples = len(dataset)
print("\n")  # Add a blank line for better readability

# Print statistics about the dataset
print_dataset_statistics(n_train_examples, convo_lens)

Number of examples: 100
First example:
{'role': 'system', 'content': 'Given a sports headline, provide the following fields in a JSON dict, where applicable: "player" (full name), "team", "sport", and "gender".'}
{'role': 'user', 'content': 'Lakers sign LeBron James to 2-year contract extension'}
{'role': 'assistant', 'content': '{"player": "LeBron James", "team": "Lakers", "sport": "basketball", "gender": "male" }'}


No errors found


Summary of dataset processing:
Num examples missing system message: 0
Num examples missing user message: 0
Total number of examples: 100
Average number of messages per example: 3.00
Average number of total tokens per example: 86.15
Average number of assistant tokens per example: 27.48
0 examples may be over the 64000 token limit and will be truncated during fine-tuning




Dataset Statistics:
- Number of training examples: 100
- Approximate billable tokens: 8615
- Default number of epochs: 3
- Estimated total billable tokens: 25845


#### Create a Train / Test Split

In [61]:
# Split the JSONL file into training and testing sets
train_test_files = split_jsonl_file(data_path)
print("\n")  # Add a blank line for better readability

# Convert file paths to strings
train_path, test_path = [str(file) for file in train_test_files]

# Print the paths of the resulting train and test files
print(f"Train file path: {train_path}")
print(f"Test file path: {test_path}")

Train data saved to: artifacts\sports_headlines_train.jsonl
Test data saved to: artifacts\sports_headlines_test.jsonl
Train set size: 80
Test set size: 20


Train file path: artifacts\sports_headlines_train.jsonl
Test file path: artifacts\sports_headlines_test.jsonl


#### Upload Our Training and Test Files

In [62]:
# Upload the training data to the OpenAI API
sports_headlines_train_file = client.files.create(
            file=open(train_path, "rb"),
            purpose="fine-tune"
            )

# Upload the training data to the OpenAI API
sports_headlines_test_file = client.files.create(
            file=open(test_path, "rb"),
            purpose="fine-tune"
            )

#### Submit Our Fine-Tuning Job

In [63]:
# Create a fine-tuning job using the uploaded training data
sports_headlines_ft_job = client.fine_tuning.jobs.create(
    model="gpt-4o-mini-2024-07-18",  # Base model to be fine-tuned
    training_file=sports_headlines_train_file.id,  # ID of the uploaded training data file
    validation_file=sports_headlines_test_file.id,  # ID of the uploaded validation (test) data file
    hyperparameters={
        "batch_size": "auto",  # Let API automatically determine batch size
        "learning_rate_multiplier": "auto",  # Auto-set learning rate multiplier
        "n_epochs": "auto",  # Automatically decide number of training epochs
    },
    suffix="sports_headlines",  # Append this to the fine-tuned model's name
    integrations=None,  # No specific integrations used
    seed=None,  # No specific random seed set for reproducibility
)
print("Fine-tuning job created successfully!")
print(sports_headlines_ft_job)
keep_running = False

Fine-tuning job created successfully!
FineTuningJob(id='ftjob-djbWbqzCBtXXWV6jSwlMxZkt', created_at=1723408446, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=[], seed=335027432, status='validating_files', trained_tokens=None, training_file='file-4U0ALDbq0fatlKa349EeuETQ', validation_file='file-Zq9XglqitaAcXvvtT2Z109cW', estimated_finish=None, integrations=[], user_provided_suffix='sports_headlines')


#### Retrieve and Print Validation Metrics

In [64]:
# Check the status of the fine-tuning job
# Continue checking until the job is complete or fails
job_id = sports_headlines_ft_job.id  
final_job = check_fine_tuning_status(client, job_id)

Current status: validating_files
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: succeeded
Job succeeded. Final job details:
FineTuningJob(id='ftjob-djbWbqzCBtXXWV6jSwlMxZkt', created_at=1723408446, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:sports-headlines:9v9iuuIi', finished_at=1723409098, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=['file-bZZvotnmT3Kfcfp1yjP9pSPb'], seed=335027432, status='succeeded', trained_tokens=20244, training_file='file-4U0ALDbq0fatlKa349EeuETQ', validation_file='file-Zq9XglqitaAcXvvtT2Z109cW', estimated_finish=None, integrations=[], u

In [65]:
# Fetch and process metrics for the sports headlines fine-tuning job
fetch_and_process_fine_tuning_metrics(client, sports_headlines_ft_job.id)


File saved as step_metrics_ft_gpt-4o-mini-2024-07-18_personal_sports-headlines_9v9iuuIi.csv


,step,train_loss,train_accuracy,valid_loss,valid_mean_token_accuracy
0,1,1.49688,0.81818,NaN,NaN
1,2,1.20318,0.90000,NaN,NaN
2,3,1.30965,0.89286,NaN,NaN
3,4,0.99305,0.91429,NaN,NaN
4,5,1.13596,0.90323,NaN,NaN
...,...,...,...,...,...
235,236,0.00009,1.00000,NaN,NaN
236,237,0.00007,1.00000,NaN,NaN
237,238,0.00025,1.00000,NaN,NaN
238,239,0.00003,1.00000,NaN,NaN


#### Use the New Model



In [66]:
# Retrieve the fine-tuning job by ID
retrieved_job = client.fine_tuning.jobs.retrieve(sports_headlines_ft_job.id)

# Print detailed information about the retrieved job
print(f"""
Retrieved Job Details:
----------------------
Job ID: {retrieved_job.id}
Status: {retrieved_job.status}
Model: {retrieved_job.model}
Created At: {retrieved_job.created_at}
Finished At: {retrieved_job.finished_at or 'Not finished yet'}
Fine-tuned Model: {retrieved_job.fine_tuned_model or 'Not available yet'}
Organization ID: {retrieved_job.organization_id}
Result Files: {', '.join(retrieved_job.result_files) if retrieved_job.result_files else 'None'}
Trained Tokens: {retrieved_job.trained_tokens or 'Not available'}
Hyperparameters:
    - Epochs: {retrieved_job.hyperparameters.n_epochs}
    - Batch Size: {retrieved_job.hyperparameters.batch_size}
    - Learning Rate Multiplier: {retrieved_job.hyperparameters.learning_rate_multiplier}
Training File: {retrieved_job.training_file}
Validation File: {retrieved_job.validation_file}
""")



Retrieved Job Details:
----------------------
Job ID: ftjob-djbWbqzCBtXXWV6jSwlMxZkt
Status: succeeded
Model: gpt-4o-mini-2024-07-18
Created At: 1723408446
Finished At: 1723409098
Fine-tuned Model: ft:gpt-4o-mini-2024-07-18:personal:sports-headlines:9v9iuuIi
Organization ID: org-SQH2HT1IvRszon9pdYwV1yvQ
Result Files: file-bZZvotnmT3Kfcfp1yjP9pSPb
Trained Tokens: 20244
Hyperparameters:
    - Epochs: 3
    - Batch Size: 1
    - Learning Rate Multiplier: 1.8
Training File: file-4U0ALDbq0fatlKa349EeuETQ
Validation File: file-Zq9XglqitaAcXvvtT2Z109cW



In [67]:
# Use the fine-tuned model to generate a completion
try:
    completion = client.chat.completions.create(
        model=retrieved_job.fine_tuned_model,  # Use the fine-tuned model
        messages=[
            {"role": "system", "content": "Given a sports headline, provide the following fields in a JSON dict, where applicable: \"player\" (full name), \"team\", \"sport\", and \"gender\"."},
            {"role": "user", "content": "2024 Olympics: Biles earns 7th gold in dominant fashion"},
        ]
    )
    
    # Print the generated response
    print("Generated Response:")
    print(completion.choices[0].message.content)
    
    # Attempt to parse and pretty-print the JSON response
    try:
        json_response = json.loads(completion.choices[0].message.content)
        print("\nFormatted JSON Response:")
        print(json.dumps(json_response, indent=2))
    except json.JSONDecodeError:
        print("\nNote: The response is not in valid JSON format.")

except Exception as e:
    print(f"An error occurred while generating the completion: {str(e)}")

Generated Response:
{"player": "Simone Biles", "team": null, "sport": "gymnastics", "gender": "female" }

Formatted JSON Response:
{
  "player": "Simone Biles",
  "team": null,
  "sport": "gymnastics",
  "gender": "female"
}


### Tool Calling: Weather Example


#### Validate the Data File

In [68]:
# Path to the JSONL data file
data_path = "./artifacts/get_current_weather.jsonl"

try:
    # Load and print the dataset
    dataset = load_and_print_dataset(data_path)
    print("\n")  # Add a blank line for better readability

    # Check for format errors in the dataset
    format_errors = check_format_errors(dataset)
    print("\n")  # Add a blank line for better readability

    # Process the dataset to extract message counts and token lengths
    n_messages, convo_lens, assistant_message_lens = process_dataset(
        dataset, 
        num_tokens_from_messages, 
        num_assistant_tokens_from_messages
    )
    print("\n")  # Add a blank line for better readability

    # Get the total number of examples in the dataset
    n_train_examples = len(dataset)

    # Print statistics about the dataset
    print_dataset_statistics(n_train_examples, convo_lens)

except FileNotFoundError:
    print(f"Error: The file {data_path} was not found.")
    sys.exit(1)
except json.JSONDecodeError:
    print(f"Error: The file {data_path} is not a valid JSON Lines file.")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")
    sys.exit(1)

Number of examples: 100
First example:
{'role': 'system', 'content': 'You are a helpful assistant that can retrieve weather information.'}
{'role': 'user', 'content': "What's the weather like in Paris?"}
{'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Paris, France", "format": "celsius"}'}}
{'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Paris, France", "temperature": 19.0, "unit": "°C", "humidity": 55, "condition": "sunny"}'}
{'role': 'assistant', 'content': "Currently in Paris, the weather is sunny with a temperature of 19.0°C and humidity at 55%. It's a pleasant day, perfect for a walk by the Seine or a visit to the Eiffel Tower."}


Found possible issues:
missing_content: 100


Summary of dataset processing:
Num examples missing system message: 0
Num examples missing user message: 0
Total number of examples: 100
Average number of messages per example: 5.00
Average number of total token

#### Create a Train/Test Split

In [69]:
# Path to the original JSONL data file
data_path = "./artifacts/get_current_weather.jsonl"

try:
    # Split the JSONL file into training and testing sets
    train_test_files = split_jsonl_file(data_path)
    print("\n")  # Add a blank line for better readability

    # Convert file paths to strings
    train_path, test_path = [str(file) for file in train_test_files]

    # Print the paths of the resulting train and test files
    print(f"Train file path: {train_path}")
    print(f"Test file path: {test_path}")

except FileNotFoundError:
    print(f"Error: The file {data_path} was not found.")
    sys.exit(1)
except ValueError as e:
    print(f"Error in splitting the file: {str(e)}")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")
    sys.exit(1)

Train data saved to: artifacts\get_current_weather_train.jsonl
Test data saved to: artifacts\get_current_weather_test.jsonl
Train set size: 80
Test set size: 20


Train file path: artifacts\get_current_weather_train.jsonl
Test file path: artifacts\get_current_weather_test.jsonl



#### Upload the Training File

In [70]:
# Upload the training data to the OpenAI API
get_current_weather_train_file = client.files.create(
            file=open("./artifacts/get_current_weather_train.jsonl", "rb"),
            purpose="fine-tune"
            ) 

# Upload the test data to the OpenAI API
get_current_data_test_file = client.files.create(
            file=open("./artifacts/get_current_weather_test.jsonl", "rb"),
            purpose="fine-tune"
            )

In [ ]:
# Checking every two hours for the the job to be done
retry_interval=7200 # every 2 hours
attempt = 1
while True:
    try:
        get_current_weather_ft_job = client.fine_tuning.jobs.create(
            model="gpt-4o-mini-2024-07-18",
            training_file=get_current_weather_train_file.id, 
            validation_file=get_current_data_test_file.id,
            hyperparameters={
                "batch_size": "auto",
                "learning_rate_multiplier": "auto",
                "n_epochs": "auto",
            },
            suffix="get-weather",
            integrations=None,
            seed=None,
        )
        print("Fine-tuning job created successfully!")
    except RateLimitError as e:
        print(f"Attempt {attempt}: Rate limit exceeded. Retrying in {retry_interval//3600} hours...")
        time.sleep(retry_interval)
        attempt += 1
    except Exception as e:
        print(f"Attempt {attempt}: An unexpected error occurred: {str(e)}")
        print(f"Retrying in {retry_interval//3600} hours...")
        time.sleep(retry_interval)
        attempt += 1


In [71]:
get_current_weather_ft_job = client.fine_tuning.jobs.create(
            model="gpt-4o-mini-2024-07-18",
            training_file=get_current_weather_train_file.id, 
            validation_file=get_current_data_test_file.id,
            hyperparameters={
                "batch_size": "auto",
                "learning_rate_multiplier": "auto",
                "n_epochs": "auto",
            },
            suffix="get-weather",
            integrations=None,
            seed=None,
        )

In [73]:
# Usage: Check the status of the fine-tuning job
try:
    # Extract the job ID from the previously created fine-tuning job
    job_id = get_current_weather_ft_job.id
    print(f"Starting to monitor fine-tuning job with ID: {job_id}")

    # Check the fine-tuning status until completion or failure
    final_job = check_fine_tuning_status(client, job_id)
    
    # Print the final job details
    print("\nFinal job details:")
    print(f"Status: {final_job.status}")
    print(f"Created at: {final_job.created_at}")
    print(f"Finished at: {final_job.finished_at}")
    print(f"Fine-tuned model: {final_job.fine_tuned_model}")
    # You can add more job details here if needed

except AttributeError:
    print("Error: 'job' object does not have 'id' attribute. "
        "Make sure the job was created successfully.")

except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

finally:
    print("Fine-tuning status check process completed.")

Starting to monitor fine-tuning job with ID: ftjob-BZESclQIpEHu6u6omnK4bwok
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: succeeded
Job succeeded. Final job details:
FineTuningJob(id='ftjob-BZESclQIpEHu6u6omnK4bwok', created_at=1723451159, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:get-weather:9vKolhs0', finished_at=1723451745, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=['file-bLCiuBkvrh4j0dKFyWkbPBLk'], seed=274768513, status='succeeded', trained_tokens=95295, training_file='file-nyk8noJx5fdab32xeqNkvbfg', validation_file='file-YQICDJhqVt8uqW9hp4a50n61', estimated_finish=None, inte

#### Use the New Model

In [74]:
# Function to simulate getting the current weather for a location
def get_current_weather(location: str, format: str = "celsius"):
    """
    Get the current weather for a given location.

    Args:
    location (str): The location to get weather for.
    format (str): The temperature format, either "celsius" or "fahrenheit". Defaults to "celsius".

    Returns:
    dict: A dictionary containing weather information.
    """
    # List of possible weather conditions
    weather_conditions = ["sunny", "partly cloudy", "cloudy", "rainy", "stormy"]
    
    # Generate random temperature and humidity
    temperature = random.uniform(-10, 35)
    humidity = random.randint(30, 90)
    
    # Convert temperature to Fahrenheit if requested
    if format.lower() == "fahrenheit":
        temperature = (temperature * 9/5) + 32
    
    # Prepare and return the weather data
    return {
        "location": location,
        "temperature": round(temperature, 1),
        "unit": "°F" if format.lower() == "fahrenheit" else "°C",
        "humidity": humidity,
        "condition": random.choice(weather_conditions)
    }


In [75]:
# Retrieve the details of a specific fine-tuning job

try:
    # Retrieve the fine-tuning job by its ID
    retrieved_job = client.fine_tuning.jobs.retrieve(get_current_weather_ft_job.id)

    # Print detailed information about the retrieved job
    print("Retrieved Job Details:")
    print(f"Job ID: {retrieved_job.id}")
    print(f"Status: {retrieved_job.status}")
    print(f"Model: {retrieved_job.model}")
    print(f"Created At: {retrieved_job.created_at}")
    print(f"Finished At: {retrieved_job.finished_at or 'Not finished yet'}")
    print(f"Fine-tuned Model: {retrieved_job.fine_tuned_model or 'Not available yet'}")
    print(f"Organization ID: {retrieved_job.organization_id}")
    print(f"Result Files: {', '.join(retrieved_job.result_files) if retrieved_job.result_files else 'None'}")
    print(f"Status: {retrieved_job.status}")
    print(f"Trained Tokens: {retrieved_job.trained_tokens}")
    print("Hyperparameters:")
    print(f"  - Epochs: {retrieved_job.hyperparameters.n_epochs}")
    print(f"  - Batch Size: {retrieved_job.hyperparameters.batch_size}")
    print(f"  - Learning Rate Multiplier: {retrieved_job.hyperparameters.learning_rate_multiplier}")

except AttributeError:
    print("Error: 'job' object does not have 'id' attribute. Make sure the job was created successfully.")
except Exception as e:
    print(f"An error occurred while retrieving the job: {str(e)}")

Retrieved Job Details:
Job ID: ftjob-BZESclQIpEHu6u6omnK4bwok
Status: succeeded
Model: gpt-4o-mini-2024-07-18
Created At: 1723451159
Finished At: 1723451745
Fine-tuned Model: ft:gpt-4o-mini-2024-07-18:personal:get-weather:9vKolhs0
Organization ID: org-SQH2HT1IvRszon9pdYwV1yvQ
Result Files: file-bLCiuBkvrh4j0dKFyWkbPBLk
Status: succeeded
Trained Tokens: 95295
Hyperparameters:
  - Epochs: 3
  - Batch Size: 1
  - Learning Rate Multiplier: 1.8


In [76]:

def get_weather_response(user_input):
    print("Step 1: Initial API call for function calling")
    # First API call to get the function call
    response = client.chat.completions.create(
        model=retrieved_job.fine_tuned_model,  
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can retrieve weather information."},
            {"role": "user", "content": user_input}
        ],
        functions=[{
            "name": "get_current_weather",
            "description": "Get the current weather for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city and country, e.g., 'London, UK'"},
                    "format": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The temperature unit to use"}
                },
                "required": ["location"]
            }
        }],
        function_call="auto"
    )
    
    # Extract and print function call details
    function_call = response.choices[0].message.function_call
    print("\nFunction Call Details:")
    print(f"Function Name: {function_call.name}")
    print(f"Arguments: {function_call.arguments}")
    
    print("\nStep 2: Simulating weather data retrieval")
    # Here you would actually call your weather API with these arguments
    # For this example, let's simulate a weather response
    function_args = json.loads(function_call.arguments)
    weather_data = get_current_weather(
        location=function_args['location'],
        format=function_args.get('format', 'celsius')
    )
    print(f"Weather Data: {json.dumps(weather_data, indent=2)}")
    
    print("\nStep 3: Second API call for human-readable response")
    # Second API call to get the final response
    final_response = client.chat.completions.create(
        model=retrieved_job.fine_tuned_model,  
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can retrieve weather information."},
            {"role": "user", "content": user_input},
            {"role": "assistant", "content": None, "function_call": function_call},
            {"role": "function", "name": "get_current_weather", "content": json.dumps(weather_data)}
        ]
    )
    
    return final_response.choices[0].message.content

# Example usage
user_query = "What's the weather like in Tokyo?"
print(f"User Query: {user_query}\n")
response = get_weather_response(user_query)
print("\nFinal Response:")
print(response)

User Query: What's the weather like in Tokyo?

Step 1: Initial API call for function calling

Function Call Details:
Function Name: get_current_weather
Arguments: {"location": "Tokyo, Japan", "format": "celsius"}

Step 2: Simulating weather data retrieval
Weather Data: {
  "location": "Tokyo, Japan",
  "temperature": 32.9,
  "unit": "\u00b0C",
  "humidity": 68,
  "condition": "stormy"
}

Step 3: Second API call for human-readable response

Final Response:
Currently in Tokyo, the weather is stormy with a temperature of 32.9°C and 68% humidity. Be cautious and stay indoors if possible.


## Integrations


### Create Our Train/Test Files

In [27]:
# File paths and data processing
file_path = "./artifacts/marv_fine_tune.jsonl"

# Split the JSONL file into train and test sets
wandb_train_test_files = split_jsonl_file(file_path)
print("\n")  # Print a blank line for better output readability

# Convert the returned file paths to strings
wandb_train_path, wandb_test_path = [str(file) for file in train_test_files]

# Print the paths of the resulting train and test files
print(f"Train file path: {wandb_train_path}")
print(f"Test file path: {wandb_test_path}")

Train data saved to: artifacts\marv_fine_tune_train.jsonl
Test data saved to: artifacts\marv_fine_tune_test.jsonl
Train set size: 80
Test set size: 20


Train file path: artifacts\marv_fine_tune_train.jsonl
Test file path: artifacts\marv_fine_tune_test.jsonl


### Upload the Train/Test Files

In [28]:
# Upload the training data to the OpenAI API
wandb_train__set_file = client.files.create(
            file=open(wandb_train_path, "rb"),
            purpose="fine-tune"
            )

# Upload the test data to the OpenAI API
wandb_test_set_file = client.files.create(
            file=open(wandb_test_path, "rb"),
            purpose="fine-tune"
            )

### Create the Fine-Tuning Job

In [29]:
# Create a fine-tuning job using the uploaded training data
wandb_params_ft_job = client.fine_tuning.jobs.create(
    model="gpt-4o-mini-2024-07-18",  # Base model to be fine-tuned
    training_file=wandb_train__set_file.id,  # ID of the uploaded training data file
    validation_file=wandb_test_set_file.id,  # ID of the uploaded validation (test) data file
    hyperparameters={
        "batch_size": "auto",  # Let API automatically determine batch size
        "learning_rate_multiplier": "auto",  # Auto-set learning rate multiplier
        "n_epochs": "auto",  # Automatically decide number of training epochs
    },
    suffix="marv_wandb_tune",  # Append this to the fine-tuned model's name
    integrations=[
        {
            "type": "wandb",
            "wandb": {
                "project": "Marv_Fun_Tune_v2",  # Replace with your actual project name
                "name": "Marv_run_001",  # Optional: Replace with your desired run name or remove
                "entity": "suspicious-cow-self",  # Optional: Replace with your entity or remove
                "tags": ["rando_tag1", "rando_tag2"]  # Optional: Replace with your desired tags or remove
            }
        }
    ],
    seed=None,  # Specific random seed set for reproducibility
)

### Check to Make Sure the Job is Done

In [30]:
# Usage: Check the status of the fine-tuning job
try:
    # Extract the job ID from the previously created fine-tuning job
    job_id = wandb_params_ft_job.id
    print(f"Starting to monitor fine-tuning job with ID: {job_id}")

    # Check the fine-tuning status until completion or failure
    final_job = check_fine_tuning_status(client, job_id)
    
    # Print the final job details
    print("\nFinal job details:")
    print(f"Status: {final_job.status}")
    print(f"Created at: {final_job.created_at}")
    print(f"Finished at: {final_job.finished_at}")
    print(f"Fine-tuned model: {final_job.fine_tuned_model}")
    # You can add more job details here if needed

except AttributeError:
    print("Error: 'job' object does not have 'id' attribute. "
        "Make sure the job was created successfully.")

except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

finally:
    print("Fine-tuning status check process completed.")

Starting to monitor fine-tuning job with ID: ftjob-WcdcO2exERwuEIK1fEcM2JBT
Current status: validating_files
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: running
Current status: succeeded
Job succeeded. Final job details:
FineTuningJob(id='ftjob-WcdcO2exERwuEIK1fEcM2JBT', created_at=1723400659, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:marv-wandb-tune:9v7hfQoT', finished_at=1723401333, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-SQH2HT1IvRszon9pdYwV1yvQ', result_files=['file-9rvOEZkwsslSW94AI2J0GcBU'], seed=32159879, status='succeeded', trained_tokens=10542, training_file='file-oeSDEG9qQLA9aGSL3wN8yLsO', validation_file=

### Head Over to Weights and Biases

https://wandb.ai/home

## Cleanup

### Delete All Fine-Tuning Jobs

In [ ]:
# List all our fine-tuning jobs
ft_jobs_list = client.fine_tuning.jobs.list()

# Filter the list to only include jobs with a status of "succeeded"
succeeded_jobs_list = [job for job in ft_jobs_list.data if job.status == "succeeded"]

print("There are a total of " + str(len(succeeded_jobs_list)) + " successful jobs\n")

print("===== Successful Jobs =====")
for job in succeeded_jobs_list:
    print(job.id)

print("\n")

# Loop through all succeeded jobs and mark the models for deletion if they exist
# If a model doesn't exist then log it and continue to the next job
# Most errors will be to delete models that don't exist because they have already been marked for deletion
# NOTE: It can take a very long time for a model to be deleted after it has been marked for deletion
# for job in succeeded_jobs_list:
#     try:
#         response = client.models.delete(job.fine_tuned_model)
#         print(f"Deleted model: {job.fine_tuned_model}")
#     except Exception as e:
#         print(f"Failed to delete model {job.fine_tuned_model}: {e}")
